# Tarea 6: Transformers aplicado a traducción y clasificación de textos
## Curso: Procesamiento de Lenguaje Natural (NLP)
## Maestría en Computación
## Instituto Tecnológico de Costa Rica (ITCR)

Medio de entrega:  TecDigital.

Entregables: Un archivo jupyter con los ejercicios resueltos.

Modo de trabajo: individual o en grupos de máximo dos personas.

**Elaborado por**:


*   Victoria Orozco Arias. Carnet $2022438528$
*   Esteban Villalobos Gómez. Carnet $9913628$

**Fecha de entrega:** 15/10/2024


---



## Ejercicio 1: Creación de un transformer para traducción de textos

Este ejercicio consiste en crear de cero un modelo de transformer (codificador-decodificador) utilizando la biblioteca de PyTorch y la arquitectura básica vista en clase y que se describe en la publicación original "Attention Is All You Need" (Vaswani et al., 2017). El modelo debe ser entrenado para realizar traducciones de texto.


El dataset a utilizar está disponible en Hugging Face en librakevin/opus_books_split 1y consiste en un conjunto de datos para realizar traducción de inglés a italiano. 


El conjunto de datos es muy simple y pequeño, contiene el texto en ingles y su traducción al italiano. Ejemplo: { "en": "There was no possibility of taking a walk that day.", "it": "I. In quel giorno era impossibile passeggiare." }. Existe una versión más completa del conjunto de datos en https://huggingface.co/datasets/Helsinki-NLP/opus_books por si lo prefieren, esta incluye traducción al español.


Deben entregar un cuaderno de Jupyter con el siguiente contenido:

### 1. Carga y pre-procesamiento de los datos

  * (5 puntos) Procese el conjunto de datos (limpiar, tokenizar y realizar lo necesario
llevar a cabo el ejercicio).

In [27]:
%%capture
!pip install --upgrade transformers datasets evaluate huggingface_hub accelerate>=0.26.0 tiktoken


In [28]:
import torch
import accelerate
import pandas as pd
from IPython.display import display
from transformers import AutoTokenizer, AutoModelForCausalLM


print(accelerate.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

1.0.1


device(type='cuda')

In [ ]:
# from huggingface_hub import login
# login()

In [29]:
splits = {'train': 'en-it/train-00000-of-00001.parquet', 'validation': 'en-it/validation-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/librakevin/opus_books_split/" + splits["train"])

In [30]:
df.head()

,translation
0,"{'en': 'Source: Project Gutenberg', 'it': 'Sou..."
1,"{'en': 'Jane Eyre', 'it': 'Jane Eyre'}"
2,"{'en': 'Charlotte Bronte', 'it': 'Charlotte Br..."
3,"{'en': 'CHAPTER I', 'it': 'PARTE PRIMA'}"
4,{'en': 'There was no possibility of taking a w...


In [31]:
df = df.drop(df.index[0])

In [32]:
df.head()

,translation
1,"{'en': 'Jane Eyre', 'it': 'Jane Eyre'}"
2,"{'en': 'Charlotte Bronte', 'it': 'Charlotte Br..."
3,"{'en': 'CHAPTER I', 'it': 'PARTE PRIMA'}"
4,{'en': 'There was no possibility of taking a w...
5,"{'en': 'We had been wandering, indeed, in the ..."


In [33]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2", model_max_length=750)

# Extract the 'en' and 'it' fields and tokenize
df['tokenized_en'] = df['translation'].apply(lambda x: tokenizer.encode(x['en'], return_tensors="pt").to(device))
df['tokenized_it'] = df['translation'].apply(lambda x: tokenizer.encode(x['it'], return_tensors="pt").to(device))

# Display the tokenized DataFrame
df.head()

,translation,tokenized_en,tokenized_it
1,"{'en': 'Jane Eyre', 'it': 'Jane Eyre'}","[[tensor(41083, device='cuda:0'), tensor(21566...","[[tensor(41083, device='cuda:0'), tensor(21566..."
2,"{'en': 'Charlotte Bronte', 'it': 'Charlotte Br...","[[tensor(24453, device='cuda:0'), tensor(11404...","[[tensor(24453, device='cuda:0'), tensor(11404..."
3,"{'en': 'CHAPTER I', 'it': 'PARTE PRIMA'}","[[tensor(41481, device='cuda:0'), tensor(314, ...","[[tensor(30709, device='cuda:0'), tensor(36, d..."
4,{'en': 'There was no possibility of taking a w...,"[[tensor(1858, device='cuda:0'), tensor(373, d...","[[tensor(40, device='cuda:0'), tensor(13, devi..."
5,"{'en': 'We had been wandering, indeed, in the ...","[[tensor(1135, device='cuda:0'), tensor(550, d...","[[tensor(14772, device='cuda:0'), tensor(23963..."


### 2. Creación del modelo

  * (20 puntos) Programe el codificador, el decodificador y todo lo necesario para implementar el transformer utilizando PyTorch.

In [ ]:
# todo

### 3. Experimentos

 * (1 punto) Separe las muestras en datos de entrenamiento y validación.
 * (5 puntos) Entrene un modelo.
 * (2 puntos) Grafique la curva de error, explique los resultados obtenidos y ajuste el modelo o el proceso de entrenamiento apropiadamente (por ejemplo el número de épocas).
 * (2 puntos) Evalúe el modelo resultante con los datos de prueba utilizando métricas como BertScore2 y METEOR3.
 * (2 puntos) Analice los resultados y presente al menos cuatro conclusiones.
 * (2 puntos) Todas las secciones del ejercicio deben estar bien documentadas.

In [ ]:
# split df into train and validate datasets, and convert them to torch tensors. 20% of the data is reserved for validation
train_df = df.sample(frac=0.8, random_state=42)
validate_df = df.drop(train_df.index)

# convert the dataframes to torch tensors
train_en = torch.cat(train_df['tokenized_en'].tolist())
train_it = torch.cat(train_df['tokenized_it'].tolist())
validate_en = torch.cat(validate_df['tokenized_en'].tolist())
validate_it = torch.cat(validate_df['tokenized_it'].tolist())
